In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import math

# Parameters
learning_rate = 0.0001 #0.001


C:\___Anaconda\envs\forTFgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\___Anaconda\envs\forTFgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\___Anaconda\envs\forTFgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\___Anaconda\envs\forTFgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Futu

In [25]:
import csv
import os
import numpy as np

num_ft = 5

ft_size = 1024
#ft_size = 4096
#version = 'pre'
version = '2019'
#version = '1718_0'
#versions = ['2019', '1718_0']
#versions = ['pre']

#인코딩중요
#라벨만들기
with open('./Answer/malSet_%s_Answer.csv'%version, 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])

#이름빼기
_Y = [y[1] for y in data]
_Y = np.array(_Y)
_Y = _Y.astype(float) # shape(-1)
_Y = [[1-y, y] for y in _Y] # shape(-1, 2)
print('Y loaded.')

_X = []
for i in range(num_ft):
    #데이터만들기
    with open('./features/ft_%s_%d_%dg.csv'%(version,ft_size,i+2), 'r', encoding='UTF-8-sig') as raw:
        lines = raw.readlines()
    data = list(csv.reader(lines))
    data = sorted(data, key=lambda x: x[0])
    #이름빼기 
    _X1 = [x[1:] for x in data]
    _X1 = np.array(_X1)
    _X1 = _X1.astype(float) # shape(-1, 1024)
    print('X%s loaded.'%i)
    _X.append(_X1)

all_set = list(zip(*_X, _Y)) # shape(10000, 6, ~)
#M_all_set = [all_set[i] for i in range(len(all_set)) if all_set[i][-1][1] == 1] # 7000
#N_all_set = [all_set[i] for i in range(len(all_set)) if all_set[i][-1][0] == 1] # 3000
#np.random.shuffle(M_all_set)
#np.random.shuffle(N_all_set)

# 악성,정상 개수 맞추기
#if len(M_all_set) > len(N_all_set):
#    M_all_set = M_all_set[:len(N_all_set)]
#else:
#    N_all_set = N_all_set[:len(M_all_set)]

#all_set = M_all_set + N_all_set

Y loaded.
X0 loaded.
X1 loaded.
X2 loaded.
X3 loaded.
X4 loaded.


In [ ]:
import csv
import os
import numpy as np

ft_size = 4096
version = 'pre'
#version = '2019'
#version = '1718_0'

#인코딩중요
#라벨만들기
#with open('./Answer/malSet_%s_Answer.csv'%version, 'r', encoding='UTF-8-sig') as raw:
with open('./Answer/malSet_%s_Answer.csv'%version, 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])
print('Y loaded.')
#이름빼기
_Y = [y[1] for y in data]
_Y = np.array(_Y)
_Y = _Y.astype(float) # shape(-1)
_Y = [[1-y, y] for y in _Y] # shape(-1, 2)

#name_Y = set([y[0] for y in data])


#데이터만들기
with open('./features/ft_%s_%d_2g.csv'%(version,ft_size), 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])
print('X1 loaded.')
#이름빼기 
_X1 = [x[1:] for x in data]
_X1 = np.array(_X1)
_X1 = _X1.astype(float) # shape(-1, 1024)

#name_X1 = set([x[0] for x in data])

#무결성검사
#name = name_X1 | name_Y
#if len(name) != len(name_Y):
#    raise Exception('이름다른게있음')

all_set = list(zip(_X1, _Y)) # shape(10000, 6, ~)

In [22]:
M_all_set = [all_set[i] for i in range(len(all_set)) if all_set[i][-1][1] == 1] # 7000
N_all_set = [all_set[i] for i in range(len(all_set)) if all_set[i][-1][0] == 1] # 3000
np.random.shuffle(M_all_set)
np.random.shuffle(N_all_set)

# 악성,정상 개수 맞추기
if len(M_all_set) > len(N_all_set):
    M_all_set = M_all_set[:len(N_all_set)]
else:
    N_all_set = N_all_set[:len(M_all_set)]

all_set = M_all_set + N_all_set

np.random.shuffle(all_set) # 6000

In [26]:
#tester
K = 1
train_set = all_set
test_set = all_set
#train_set = [all_set[i] for i in range(len(all_set)) if (i-K)%5 != 0]
#test_set = [all_set[i] for i in range(len(all_set)) if (i-K)%5 == 0]


#train_set = all_set[len(all_set)//5:] # 4800
#test_set = all_set[:len(all_set)//5] # 1200

train_Xs = []
test_Xs = []
for i in range(num_ft):
    train_X = np.array([T[i] for T in train_set])
    train_Xs.append(train_X)
    test_X = np.array([T[i] for T in test_set])
    test_Xs.append(test_X)
    
train_Y = np.array([T[-1] for T in train_set])
test_Y = np.array([T[-1] for T in test_set])
    
M_test_Xs = []
N_test_Xs = []
for i in range(num_ft):
    M_test_X = np.array([T[i] for T in test_set if T[-1][1] == 1])# label의 악성이면
    M_test_Xs.append(M_test_X)
    N_test_X = np.array([T[i] for T in test_set if T[-1][0] == 1])# label의 정상이면
    N_test_Xs.append(N_test_X)
    
M_test_Y = np.array([T[-1] for T in test_set if T[-1][1] == 1])# label의 악성이면
N_test_Y = np.array([T[-1] for T in test_set if T[-1][0] == 1])# label의 정상이면

In [5]:
num_model = 5
num_design = 1

with tf.device('/gpu:0'):
    Xs = []
    for i in range(num_ft):
        X = tf.placeholder(tf.float32, [None, ft_size])
        Xs.append(X)
    Y = tf.placeholder(tf.float32, [None, 2])
    keep_prob = tf.placeholder(tf.float32)
    
    
    #models = [[4096,2000,1000,500,250,100,2]]
    #models = [[1024,500,250,100,2]]
    models = [[1024,500,250,100,2], \
              [1024,500,2], \
              [1024,800,400,2], \
              [1024,600,400,260,180,120,80,2], \
              [1024,400,300,200,100,2]]
    
    W = [None] * num_model
    b = [None] * num_model
    L = [None] * num_model
    for i in range(num_model):
        W[i] = [None] * (len(models[i%num_design])-1)
        b[i] = [None] * (len(models[i%num_design])-1)
        L[i] = [None] * (len(models[i%num_design])-1)
    
    hypothesis = [None] * num_model
    cost = [None] * num_model
    optimizer = [None] * num_model
    l2reg = [None] * num_model
    
    for i in range(num_model):
        for j in range(len(models[i%num_design])-1):
            W[i][j] = tf.get_variable('w%d_%d'%(i,j), shape=[models[i%num_design][j],models[i%num_design][j+1]], \
                                      initializer=tf.contrib.layers.xavier_initializer())
            b[i][j] = tf.Variable(tf.random_normal([models[i%num_design][j+1]]), name='w%d_%d'%(i,j))
        L[i][0] = tf.nn.relu(tf.matmul(Xs[i%num_ft], W[i][0]) + b[i][0])
        L[i][0] = tf.nn.dropout(L[i][0], keep_prob=keep_prob)
        for j in range(len(models[i%num_design])-3):
            L[i][j+1] = tf.nn.relu(tf.matmul(L[i][j], W[i][j+1]) + b[i][j+1])
            L[i][j+1] = tf.nn.dropout(L[i][j+1], keep_prob=keep_prob)
        hypothesis[i] = tf.matmul(L[i][-2], W[i][-1]) + b[i][-1]
        
        l2reg[i] = tf.reduce_sum([0.001 * tf.reduce_sum(tf.square(W[i][j])) for j in range(len(W[i]))])
        cost[i] = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis[i], labels=Y)) #+ l2reg[i]
        optimizer[i] = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost[i])
   

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
param_list = []
for i in range(num_model):
    for j in range(len(models[i%num_design])-1):
        param_list.append(W[i][j])
        param_list.append(b[i][j])
        
    
saver = tf.train.Saver(param_list)

In [6]:
training_epochs = 200 #15
batch_size = 100
new_or_load = 0


# Training
sess = tf.Session()
if new_or_load == 0:
    sess.run(tf.global_variables_initializer())
else:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, './tf_model_%s_%d/tf_model.ckpt'%(ft_size,num_ft))
    saver.restore(sess, './tf_model_test/tf_model.ckpt')


In [24]:

avg_costs = []
# Training cycle
for epoch in range(training_epochs):
    for i in range(num_model):
        avg_cost = 0
        avg_costs.append(avg_cost)
    total_batch = math.ceil(len(train_Y) / batch_size)

    for i in range(total_batch):
        batch_xss = []
        for j in range(num_ft):
            batch_xs = train_Xs[j][i*batch_size:(i+1)*batch_size]
            batch_xss.append(batch_xs)
        batch_ys = train_Y[i*batch_size:(i+1)*batch_size]
        
        
        for j in range(num_model):
            feed_dict = {Xs[j%num_ft] : batch_xss[j%num_ft], Y : batch_ys, keep_prob : 0.7}
            c, _ = sess.run([cost[j], optimizer[j]], feed_dict=feed_dict)
            avg_costs[j] += c * len(batch_ys) #뭉치갯수로 곱하기
        #time.sleep(0.001)
    
    #saver.save(sess, './tf_model/tf_model.ckpt')
    for i in range(num_model):
        avg_costs[i] /= len(train_Y) #전체개수로 나누기
    
    if (epoch+1)%10 == 0:
        print('Epoch: %03d\t'%(epoch+1), end='')
        for i in range(num_model):
            print('%.3f'%avg_costs[i], end='  ')
            pass
        print()
        
print(len(train_Y),' 개 학습완료')

Epoch: 010	0.323  0.324  0.330  0.338  0.349  
Epoch: 020	0.277  0.274  0.289  0.301  0.301  
Epoch: 030	0.230  0.244  0.253  0.265  0.261  
Epoch: 040	0.202  0.213  0.229  0.232  0.239  
Epoch: 050	0.177  0.182  0.202  0.213  0.218  
Epoch: 060	0.163  0.155  0.185  0.189  0.199  
Epoch: 070	0.127  0.151  0.174  0.171  0.184  
Epoch: 080	0.119  0.129  0.153  0.167  0.168  
Epoch: 090	0.115  0.122  0.144  0.140  0.157  
Epoch: 100	0.094  0.108  0.127  0.130  0.132  
Epoch: 110	0.093  0.098  0.129  0.114  0.131  
Epoch: 120	0.085  0.087  0.110  0.105  0.114  
Epoch: 130	0.082  0.076  0.098  0.112  0.120  
Epoch: 140	0.075  0.072  0.088  0.097  0.110  
Epoch: 150	0.067  0.068  0.081  0.090  0.101  
Epoch: 160	0.065  0.066  0.082  0.079  0.096  
Epoch: 170	0.065  0.057  0.075  0.080  0.100  
Epoch: 180	0.050  0.058  0.079  0.080  0.081  
Epoch: 190	0.061  0.053  0.072  0.083  0.083  
Epoch: 200	0.044  0.046  0.073  0.073  0.079  
5954  개 학습완료


In [27]:
# TEST
sum_hypo = hypothesis[0]*0.64 + \
           hypothesis[1]*3.13 + \
           hypothesis[2]*0.87 + \
           hypothesis[3]*0.34 + \
           hypothesis[4]*1.56
    
is_correct = tf.equal(tf.math.argmax(sum_hypo, 1), tf.math.argmax(Y, 1))
#is_correct = tf.equal(tf.math.argmax(tf.reduce_sum(hypothesis, axis=0), 1), tf.math.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))


total_batch = math.ceil(len(test_Y) / batch_size)
total_acc = 0
for i in range(total_batch):
    batch_xss = []
    for j in range(num_ft):
        batch_xs = test_Xs[j][i*batch_size:(i+1)*batch_size]
        batch_xss.append(batch_xs)
    batch_ys  = test_Y[i*batch_size:(i+1)*batch_size]
    
    feed_dict = {}
    for i in range(num_ft):
        feed_dict[Xs[i]] = batch_xss[i]
    feed_dict[Y] = batch_ys
    feed_dict[keep_prob] = 1
    
    one_acc = accuracy.eval(session=sess, feed_dict=feed_dict)
    total_acc += one_acc * len(batch_ys)
norm_det = total_acc/len(test_Y) # 정상을 정상으로 악성을 악성으로 예측
print("정탐율 : %0.4f"%(norm_det))


total_batch = math.ceil(len(M_test_Y) / batch_size)
total_acc = 0
for i in range(total_batch):
    batch_xss = []
    for j in range(num_ft):
        batch_xs = M_test_Xs[j][i*batch_size:(i+1)*batch_size]
        batch_xss.append(batch_xs)
    batch_ys  = M_test_Y[i*batch_size:(i+1)*batch_size]
    
    feed_dict = {}
    for i in range(num_ft):
        feed_dict[Xs[i]] = batch_xss[i]
    feed_dict[Y] = batch_ys
    feed_dict[keep_prob] = 1
    
    one_acc = accuracy.eval(session=sess, feed_dict=feed_dict)
    total_acc += one_acc * len(batch_ys)
not_det = 1 - total_acc/len(M_test_Y) # 악성코드들중에서 정상코드로 예측
print("미탐율 : %0.4f"%(not_det))


total_batch = math.ceil(len(N_test_Y) / batch_size)
total_acc = 0
for i in range(total_batch):
    batch_xss = []
    for j in range(num_ft):
        batch_xs = N_test_Xs[j][i*batch_size:(i+1)*batch_size]
        batch_xss.append(batch_xs)
    batch_ys  = N_test_Y[i*batch_size:(i+1)*batch_size]
    
    feed_dict = {}
    for i in range(num_ft):
        feed_dict[Xs[i]] = batch_xss[i]
    feed_dict[Y] = batch_ys
    feed_dict[keep_prob] = 1
    
    one_acc = accuracy.eval(session=sess, feed_dict=feed_dict)
    total_acc += one_acc * len(batch_ys)
over_det = 1 - total_acc/len(N_test_Y) # 정상코드들중에서 악성코드로 예측
print("과탐율 : %0.4f"%(over_det))

det_rate = norm_det - (over_det*0.6 + not_det*0.4)
print("탐지율 : %0.4f"%(det_rate))

print(len(test_Y),' 개 테스트완료')


#r = rd.randint(0, len(test_X) - 1)
#print('Label:', sess.run(tf.argmax(test_Y[r:r+1], 1)))

#feed_dict = {X: test_X[r:r+1], keep_prob: 1}
#print('Prediction:', sess.run(tf.argmax(hypothesis, 1), feed_dict=feed_dict))

정탐율 : 0.9391
미탐율 : 0.0400
과탐율 : 0.0922
탐지율 : 0.8678
10000  개 테스트완료


In [9]:
#학습한거 저장하기
#with open('./tf_model_%s_%d/study.txt'%(ft_size,num_ft), 'a') as f:
#    f.write('%s\n'%version)
with open('./tf_model_test/study.txt', 'a') as f:
    f.write('%s\n'%version)


#saver.save(sess, './tf_model/tf_model.ckpt')
saver.save(sess, './tf_model_test/tf_model.ckpt')
#saver.save(sess, './tf_model_%s_%d/tf_model.ckpt'%(ft_size,num_ft))

'./tf_model_test/tf_model.ckpt'

In [19]:
sess.close()